# sun397 Ablations

In [12]:
import sys
import os

if sys.platform == "darwin":
    ROOT = '/Users/rohan/repositories/11785-f23-prj/faceted_visualization'
else:
    ROOT = '/apps/11785-f23-prj/faceted_visualization'

LIBRARY_HOME = os.path.join(ROOT, 'visualizer')
PROBE_HOME = os.path.join(ROOT, "probes", "logs", "sun397")
LOG_PATH = os.path.join(ROOT, "logs")

sys.path.append(ROOT)
sys.path.append(LIBRARY_HOME)

In [13]:
%cd /apps/11785-f23-prj

/apps/11785-f23-prj


In [14]:
import faceted_visualization.visualizer.main
import faceted_visualization.visualizer.render
import faceted_visualization.visualizer.hook
import faceted_visualization.visualizer.image
import faceted_visualization.visualizer.helpers
import faceted_visualization.visualizer.wb
import faceted_visualization.visualizer.constants as constants
import wandb
import pprint
import clip
import os
import logging
import datetime
import torch

In [15]:
logging.basicConfig(force=True, level="INFO",
                    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
logger = logging.getLogger()
init_stream = True
init_filehandler = True
for handler in logger.handlers:
    if isinstance(handler, logging.StreamHandler):
        init_stream = False
    if isinstance(handler, logging.FileHandler):
        init_filehandler = False
if init_stream:
    consoleHandler = logging.StreamHandler(sys.stdout)
    logger.addHandler(consoleHandler)
if init_filehandler and sys.platform != "darwin":
    fileHandler = logging.FileHandler(os.path.join(LOG_PATH, f"{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.log"))
    fileHandler.setFormatter(
        logging.Formatter(fmt="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s"))
    logger.addHandler(fileHandler)

In [16]:
generated_random_seed = faceted_visualization.visualizer.helpers.random_seed(8)
logger.info("Seed = %s", generated_random_seed)

2023-12-05 22:05:06,429 [MainThread  ] [INFO ]  Seed = 80119365


In [17]:
parameter_dict = {
    constants.NUMBER_OF_ITERATIONS: {
        'values': [512]
    },
    constants.MODEL: {
        'values': ['RN50x4']
    },
    constants.LINEAR_PROBE_LAYER: {
        'values': [
            'layer1_0_conv3',
            'layer1_0_relu3',
            'layer1_3_conv3',
            'layer1_3_relu3',
            'layer2_5_conv3',
            'layer2_5_relu3',
            'layer3_9_conv3',
            'layer3_9_relu3'
            
        ]
    },
    constants.VISUALIZATION_LAYER: {
        'values': [
            'layer4_5_conv3'
        ]
    },
    constants.OPTIMIZER: {
        'values': [
            'AdamW'
        ]
    },
    constants.LEARNING_RATE: {
        'values': [
            0.05
        ]
    },
    constants.CHANNEL: {
        'values': [
            1, 128, 512, 1024, 2048, 2560  
        ]
    },
    constants.OBJECTIVE: {
        'values': [
            'neuron',
            'channel'
        ]
    },
    constants.USE_TRANSFORMS: {
        'values': [
            True, False
        ]
    },
    constants.USE_STD_TRANSFORMS: {
        'values': [
            True
        ]
    },
    constants.RANDOM_SEED: {
        'values': [
            generated_random_seed
        ]
    }
}
sweep_config = {
    'method': 'grid',
    'parameters': parameter_dict

}
logger.info("Sweep Configuration = \n %s", pprint.pformat(sweep_config))

2023-12-05 22:05:28,331 [MainThread  ] [INFO ]  Sweep Configuration = 
 {'method': 'grid',
 'parameters': {'channel': {'values': [1, 128, 512, 1024, 2048, 2560]},
                'learning_rate': {'values': [0.05]},
                'linear_probe_layer': {'values': ['layer1_0_conv3',
                                                  'layer1_0_relu3',
                                                  'layer1_3_conv3',
                                                  'layer1_3_relu3',
                                                  'layer2_5_conv3',
                                                  'layer2_5_relu3',
                                                  'layer3_9_conv3',
                                                  'layer3_9_relu3']},
                'model': {'values': ['RN50x4']},
                'num_iterations': {'values': [512]},
                'objective': {'values': ['neuron', 'channel']},
                'optimizer': {'values': ['AdamW']},
                'ran

In [18]:
if torch.cuda.is_available():
    DEVICE = "cuda:1"
else:
    DEVICE = "cpu"
logger.info("Selected Device = %s", DEVICE)

2023-12-05 22:05:30,203 [MainThread  ] [INFO ]  Selected Device = cuda:1


In [19]:
def perform_sweep(config=None):
    with wandb.init(config) as run:
        current_sweep_config = wandb.config
        logger.info('Starting sweep [ run  %s ] with config = \n%s', run.id, current_sweep_config)
        faceted_visualization.visualizer.helpers.set_seed(current_sweep_config[constants.RANDOM_SEED])
        model, clip_transforms = faceted_visualization.visualizer.main.get_model(current_sweep_config[constants.MODEL],
                                                                                 device=DEVICE)
        model_hook = faceted_visualization.visualizer.hook.register_hooks(model)

        if current_sweep_config[constants.MODEL].casefold() == 'RN50x4'.casefold():
            image_h, image_w = 288, 288
            probe_location = os.path.join(PROBE_HOME, 
                                          current_sweep_config[constants.MODEL],
                                          current_sweep_config[constants.LINEAR_PROBE_LAYER], "version_4_20", "model_checkpoint.pth")
        else:
            image_h, image_w = 224, 224
            probe_location = os.path.join(PROBE_HOME,
                                          current_sweep_config[constants.MODEL] + "_" + current_sweep_config[
                                              constants.LINEAR_PROBE_LAYER], "version_1", "model_checkpoint.pth")
        params, image_f = faceted_visualization.visualizer.image.generate_img(h=image_h, w=image_w,
                                                                              decorrelate=True,
                                                                              fft=True,
                                                                              device=DEVICE)
        probe_weights = faceted_visualization.visualizer.main.get_probe_weights(model_location=probe_location,
                                                                                device=DEVICE)
        if current_sweep_config[constants.OPTIMIZER].casefold() == "ADAM".casefold():
            optimizer = torch.optim.Adam(params=params, lr=current_sweep_config[constants.LEARNING_RATE])
        elif current_sweep_config[constants.OPTIMIZER].casefold() == "ADAMW".casefold():
            optimizer = torch.optim.AdamW(params=params, lr=current_sweep_config[constants.LEARNING_RATE])
        elif current_sweep_config[constants.OPTIMIZER].casefold() == "SGD".casefold():
            optimizer = torch.optim.SGD(params=params, lr=current_sweep_config[constants.LEARNING_RATE])
        else:
            logger.error("Unknown optimizer type [ %s ] provided", current_sweep_config[constants.OPTIMIZER])
            raise RuntimeError(f"Unknown optimizer type [ {current_sweep_config[constants.OPTIMIZER]} ] provided")

        transforms = faceted_visualization.visualizer.image.consolidate_transforms(
            use_clip_transforms=current_sweep_config[constants.USE_TRANSFORMS],
            use_standard_transforms=current_sweep_config[constants.USE_STD_TRANSFORMS],
            clip_transforms=clip_transforms
        )

        logger.info("Final list of transforms = %s", transforms.transforms if transforms is not None else "NA")

        wandb_object = faceted_visualization.visualizer.wb.WandB(
            is_sweep=True,
            enabled=True,
            api_key=None,
            entity=None,
            run_name=None,
            project=None,
            config=None,
        )
        
        if current_sweep_config[constants.OBJECTIVE] == "neuron":
            image_name = f"{current_sweep_config[constants.VISUALIZATION_LAYER]}:{current_sweep_config[constants.CHANNEL]}_mid_neuron"
        else:
            image_name = f"{current_sweep_config[constants.VISUALIZATION_LAYER]}:{current_sweep_config[constants.CHANNEL]}"
            
        faceted_visualization.visualizer.render.optimize(
            num_iterations=current_sweep_config[constants.NUMBER_OF_ITERATIONS],
            transforms=transforms,
            image_function=image_f,
            model=model,
            channel=current_sweep_config[constants.CHANNEL],
            objective=current_sweep_config[constants.OBJECTIVE],
            layer=current_sweep_config[constants.VISUALIZATION_LAYER],
            linear_probe_layer=current_sweep_config[constants.LINEAR_PROBE_LAYER],
            probe_weights=probe_weights,
            optimizer=optimizer,
            model_hook=model_hook,
            neuron_x=current_sweep_config.get(constants.NEURON_X, None),
            neuron_y=current_sweep_config.get(constants.NEURON_Y, None),
            wandb_object=wandb_object,
            run_id=run.id,
            device=DEVICE,
            image_name=image_name
            )

        logger.info("Completed run [ %s ]", run.id)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="idl-project-sun397", entity="rohanprasad")
logger.info("Sweep ID = %s", sweep_id)
wandb.agent(sweep_id, function=perform_sweep)

2023-12-05 22:05:39,732 [MainThread  ] [INFO ]  Sweep ID = kzps3xpf


Create sweep with ID: kzps3xpf
Sweep URL: https://wandb.ai/rohanprasad/idl-project-sun397/sweeps/kzps3xpf


wandb: Agent Starting Run: b2msacpp with config:
wandb: 	channel: 1
wandb: 	learning_rate: 0.05
wandb: 	linear_probe_layer: layer1_0_conv3
wandb: 	model: RN50x4
wandb: 	num_iterations: 512
wandb: 	objective: neuron
wandb: 	optimizer: AdamW
wandb: 	random_seed: 80119365
wandb: 	use_std_transforms: True
wandb: 	use_transforms: True
wandb: 	visualization_layer: layer4_5_conv3


2023-12-05 22:05:51,971 [Thread-584 (] [INFO ]  Starting sweep [ run  b2msacpp ] with config = 
{'channel': 1, 'learning_rate': 0.05, 'linear_probe_layer': 'layer1_0_conv3', 'model': 'RN50x4', 'num_iterations': 512, 'objective': 'neuron', 'optimizer': 'AdamW', 'random_seed': 80119365, 'use_std_transforms': True, 'use_transforms': True, 'visualization_layer': 'layer4_5_conv3'}
2023-12-05 22:05:51,972 [Thread-584 (] [INFO ]  Setting random seed to 80119365
2023-12-05 22:05:51,974 [Thread-584 (] [INFO ]  Loading CLIP model [ RN50x4 ].
2023-12-05 22:05:56,408 [Thread-584 (] [INFO ]  Finished loading model [ RN50x4 ]
2023-12-05 22:05:56,409 [Thread-584 (] [INFO ]  Finished loading transforms [ Compose(
    Resize(size=288, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(288, 288))
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
) ]
2023-12-05 22:05:56,410 [Thread-584 (] [INFO ]  Registering hooks.
2023-12-05 22:05:

loss,█▅▃▂▁▁▁
loss,-2.18164


...

wandb: Agent Starting Run: 26kcc0en with config:
wandb: 	channel: 1
wandb: 	learning_rate: 0.05
wandb: 	linear_probe_layer: layer2_5_relu3
wandb: 	model: RN50x4
wandb: 	num_iterations: 512
wandb: 	objective: neuron
wandb: 	optimizer: AdamW
wandb: 	random_seed: 80119365
wandb: 	use_std_transforms: True
wandb: 	use_transforms: False
wandb: 	visualization_layer: layer4_5_conv3


2023-12-05 22:22:37,848 [Thread-647 (] [INFO ]  Starting sweep [ run  26kcc0en ] with config = 
{'channel': 1, 'learning_rate': 0.05, 'linear_probe_layer': 'layer2_5_relu3', 'model': 'RN50x4', 'num_iterations': 512, 'objective': 'neuron', 'optimizer': 'AdamW', 'random_seed': 80119365, 'use_std_transforms': True, 'use_transforms': False, 'visualization_layer': 'layer4_5_conv3'}
2023-12-05 22:22:37,850 [Thread-647 (] [INFO ]  Setting random seed to 80119365
2023-12-05 22:22:37,851 [Thread-647 (] [INFO ]  Loading CLIP model [ RN50x4 ].
2023-12-05 22:22:42,325 [Thread-647 (] [INFO ]  Finished loading model [ RN50x4 ]
2023-12-05 22:22:42,326 [Thread-647 (] [INFO ]  Finished loading transforms [ Compose(
    Resize(size=288, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(288, 288))
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
) ]
2023-12-05 22:22:42,328 [Thread-647 (] [INFO ]  Registering hooks.
2023-12-05 22:22

loss,██▁▅▃▅▄
loss,-4.78906


wandb: Agent Starting Run: 0qg8g524 with config:
wandb: 	channel: 1
wandb: 	learning_rate: 0.05
wandb: 	linear_probe_layer: layer2_5_relu3
wandb: 	model: RN50x4
wandb: 	num_iterations: 512
wandb: 	objective: channel
wandb: 	optimizer: AdamW
wandb: 	random_seed: 80119365
wandb: 	use_std_transforms: True
wandb: 	use_transforms: True
wandb: 	visualization_layer: layer4_5_conv3


2023-12-05 22:23:24,001 [Thread-650 (] [INFO ]  Starting sweep [ run  0qg8g524 ] with config = 
{'channel': 1, 'learning_rate': 0.05, 'linear_probe_layer': 'layer2_5_relu3', 'model': 'RN50x4', 'num_iterations': 512, 'objective': 'channel', 'optimizer': 'AdamW', 'random_seed': 80119365, 'use_std_transforms': True, 'use_transforms': True, 'visualization_layer': 'layer4_5_conv3'}
2023-12-05 22:23:24,003 [Thread-650 (] [INFO ]  Setting random seed to 80119365
2023-12-05 22:23:24,004 [Thread-650 (] [INFO ]  Loading CLIP model [ RN50x4 ].
2023-12-05 22:23:28,475 [Thread-650 (] [INFO ]  Finished loading model [ RN50x4 ]
2023-12-05 22:23:28,477 [Thread-650 (] [INFO ]  Finished loading transforms [ Compose(
    Resize(size=288, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(288, 288))
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
) ]
2023-12-05 22:23:28,478 [Thread-650 (] [INFO ]  Registering hooks.
2023-12-05 22:23